PART 1

In [24]:
import pandas as p

'''
    1. mpg:           continuous
    2. cylinders:     multi-valued discrete
    3. displacement:  continuous
    4. horsepower:    continuous
    5. weight:        continuous
    6. acceleration:  continuous
    7. model year:    multi-valued discrete
    8. origin:        multi-valued discrete
    9. car name:      string (unique for each instance)
'''

def discretizer(frame, continousVariables):
    for var in continousVariables:
        frame[var] = p.cut(data[var], bins= [0, data[var].median(), data[var].max()], labels= ['low', 'high'])
    return frame

conVars = ['mpg', 'displacement', 'horsepower', 'weight', 'acceleration']

data = p.read_csv('auto-mpg.csv')

data = discretizer(data, conVars)
data.astype('category')
data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,low,8,high,high,high,low,70,1,chevrolet chevelle malibu
1,low,8,high,high,high,low,70,1,buick skylark 320
2,low,8,high,high,high,low,70,1,plymouth satellite
3,low,8,high,high,high,low,70,1,amc rebel sst
4,low,8,high,high,high,low,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
94,high,4,low,low,low,low,74,2,fiat 124 tc
95,high,4,low,low,low,high,74,3,honda civic
96,high,4,low,low,low,high,74,3,subaru
97,high,4,low,low,low,high,74,2,fiat x1.9


PART 2

In [27]:
from pgmpy.estimators import PC

# estimatedDAG = PC(data.loc[:, data.columns != 'car name'])
#estimatedDAG.estimate().edges()
est = PC(data)
skel, seperating_sets = est.build_skeleton(significance_level=0.01)
print("Undirected edges: ", skel.edges())

pdag = est.skeleton_to_pdag(skel, seperating_sets)
print("PDAG edges:       ", pdag.edges())

model = pdag.to_dag()
print("DAG edges:        ", model.edges())





c:\Users\Razi's good laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\estimators\CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):
c:\Users\Razi's good laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\pgmpy\estimators\CITests.py:548: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for z_state, df in data.groupby(Z):


TypeError: 'int' object is not iterable

PART 2B

In [30]:
from pgmpy.estimators import HillClimbSearch as hcs
from pgmpy.estimators import BDeuScore, K2Score, BicScore
climb = hcs(data)
bestHCModel = climb.estimate(scoring_method=BicScore(data))
print(bestHCModel.edges())











  0%|          | 7/1000000 [00:01<42:56:12,  6.47it/s]

[('mpg', 'horsepower'), ('cylinders', 'displacement'), ('cylinders', 'weight'), ('cylinders', 'mpg'), ('displacement', 'origin'), ('displacement', 'model year'), ('horsepower', 'acceleration')]


In [31]:

bestBDSHCModel = climb.estimate(scoring_method=BDeuScore(data))
print(bestBDSHCModel.edges())





















  0%|          | 18/1000000 [00:01<26:51:05, 10.34it/s]

[('mpg', 'weight'), ('mpg', 'model year'), ('cylinders', 'weight'), ('cylinders', 'mpg'), ('cylinders', 'horsepower'), ('cylinders', 'origin'), ('cylinders', 'acceleration'), ('cylinders', 'model year'), ('displacement', 'cylinders'), ('displacement', 'car name'), ('displacement', 'horsepower'), ('displacement', 'weight'), ('displacement', 'origin'), ('displacement', 'mpg'), ('horsepower', 'mpg'), ('horsepower', 'acceleration'), ('horsepower', 'weight'), ('weight', 'origin')]


In [32]:
bestK2HCModel = climb.estimate(scoring_method=K2Score(data))
print(bestK2HCModel.edges())





















  0%|          | 18/1000000 [00:02<31:19:04,  8.87it/s]

[('mpg', 'horsepower'), ('mpg', 'model year'), ('cylinders', 'displacement'), ('cylinders', 'weight'), ('cylinders', 'mpg'), ('cylinders', 'horsepower'), ('cylinders', 'car name'), ('cylinders', 'acceleration'), ('cylinders', 'model year'), ('displacement', 'origin'), ('displacement', 'mpg'), ('displacement', 'car name'), ('horsepower', 'acceleration'), ('horsepower', 'car name'), ('weight', 'mpg'), ('weight', 'car name'), ('acceleration', 'car name'), ('origin', 'car name')]
